# Generating and Implementing a Secure RSA Cryptosystem with Safe Prime Numbers

We want to do the following in this project:

#### 1. Generating RSA Keys

1. Select two 600-digit safe primes $p$ and $q$, and compute:
   $$ n = p \times q $$

2. Calculate Euler’s totient function:
   $$ \varphi(n) = (p - 1)(q - 1) $$

3. Choose a public exponent $e$ and compute the private exponent $d$:
   $$ d = e^{-1} \mod \varphi(n) $$

#### 2. Primality Testing

Verify whether the primes are strong pseudoprimes.

#### 3 Encrypting a Message

1. Convert plaintext into a numeric format.
2. Encrypt using RSA encryption formula:
   $$ c = m^e \mod n $$

#### 4. Decrypting the Message

1. Compute plaintext using RSA decryption formula:
   $$ m = c^d \mod n $$
2. Convert the result back into readable text.


### Generating the keys

We define a function below that generates a safe prime with exactly the specified number of digits. this function calculates the range for numbers with that many digits and then derives the corresponding range for a candidate $\ell$ such that $p = 2\ell + 1$ lies within the desired bounds. It randomly selects $\ell$, verifies that both $\ell$ and $p$ are prime, and returns $p$ once a valid safe prime is found. In our case we will generate primes with 600 digits.

In [102]:
import random
from sympy import isprime

def generate_safe_prime(digits):
    """
    Generates a safe prime with exactly 'digits' digits.
    A safe prime p is one where p is prime and (p - 1)//2 is also prime.
    
    We want p = 2l + 1 to be in the interval [10^(digits-1), 10^digits - 1].
    Then, l must be in [ (10^(digits-1) - 1)//2, (10^digits - 1)//2 ].
    """
    lower_bound = 10**(digits - 1)         # smallest number with 'digits' digits
    upper_bound = 10**digits - 1             # largest number with 'digits' digits
    
    # Compute bounds for candidate l
    l_lower = (lower_bound - 1) // 2
    l_upper = (upper_bound - 1) // 2

    while True:
        # Pick a random candidate l in the computed range.
        l = random.randint(l_lower, l_upper)
        if not isprime(l):
            continue  # l must be prime
        
        # Compute p = 2l + 1
        p = 2 * l + 1
        
        # Ensure p is in the required digit range.
        if p < lower_bound or p > upper_bound:
            continue
        
        # Check if p is prime.
        if isprime(p):
            return p


 below we will generate two distinct 600-digit safe primes $p$ and $q$.

In [245]:
# Generate a safe prime with exactly 600 digits.
p = generate_safe_prime(600)
q = generate_safe_prime(600)
print("First 600-digit safe prime p is:")
print(p)
print("Second 600-digit safe prime q is:")
print(q)

First 600-digit safe prime p is:
439573107123443334298102516300624473953295060862524822701138949569095925698442704396408486038576164418316495006075544245060464083683536745284383531984961923921752810022215387303376481571844822051749277519254055864472959338143239596103748968919246924518273080289447196381169547300137817622017556127950988262379939487813373932918749457265360242307035379653995804116767136327074289294940279762743467708106230302315917377346995078739368063983799583393715537631076793855440153124527092592814013954493192622953121132700915827401391387357238338271917430670917508477818002130657812031417413317833983471562859
Second 600-digit safe prime q is:
88002025105422468143535926055537019691560072883847244101648878794756826370408573897901850556789815705730933074765500182567270388129093633610390639961656363286496986152974041499496078682726963534461044404871918560973269709817199511760702705892605915744880759956241157085269642550906796847438682300236407823921182202496255561962057329

We will then compute the RSA modulus $n $ as the product of the two safe primes $ p $ and $ q $, calculates Euler's totient $\phi(n)$ as $(p-1)(q-1)$, and then set the public exponent $ e $ to 65537.

In [246]:
# Compute the RSA modulus n = p * q
n = p * q

# Calculate Euler's totient function φ(n) = (p - 1) * (q - 1)
phi_n = (p - 1) * (q - 1)

# Choose a public exponent, commonly 65537
e = 65537

Next, we define a function that implements the extended Euclidean algorithm. In summary, it computes the greatest common divisor (gcd) of two integers $a$ and $b$ and finds integers $u$ and $v$ such that:

$$a\cdot u + b\cdot v = \gcd(a,b)$$

It does this by iteratively updating remainders and corresponding coefficients until the remainder reaches zero.

In [247]:
#  Extended Euclidean Algorithm
def Extended_Euclidean_Algorithm(a, b):
    if a == 0 or b == 0:
        return "Error"

    r0, r1 = a, b
    s0, s1 = 1, 0
    t0, t1 = 0, 1

    while r1 > 0:
        q = r0 // r1
        r = r0 - q * r1
        r0, r1 = r1, r

        s = s0 - q * s1
        s0, s1 = s1, s

        t = t0 - q * t1
        t0, t1 = t1, t

    gcd = r0
    u = s0
    v = t0
    return gcd, u, v

We then use the Extended Euclidean Algorithm to compute the modular inverse of $a$ modulo $n$. Particularly, our function calculates Bob's private key $d$ as the inverse of the public exponent $e$ modulo $\phi(n)$, ensuring that $e \times d \equiv 1 \pmod{\phi(n)}$. The result is then printed as Bob's private key.

In [250]:
def inverse_mod_n(e, phi_n):
    gcd, u, v = Extended_Euclidean_Algorithm(e, phi_n)
    if gcd == 1:
        return u % phi_n
    else:
        return "The inverse does not exist."  # a and n are not coprime

d = inverse_mod_n(e, phi_n)
d

2049117692908158592266173263750999142939392437736410735220857363462005049730584235199432681598025242125542095531485981122866036777793628556302077447942377764987037618156388019269078778012633814739916127672589682460406809745818025428713128418462373150967717273031907252102782838703991533313713019575541168510590912275100204628354945404241682194393497553013917294413220750833536491352792182474931591201968042490630275977932015548698768883037125948816960575137074875281694582258768728413914959301318821749644917497790187501873566681656876772344827210077809160200974076270740627813862112631068969619526921608581030377991709220604005312607668076005243518726792048103449981613230746156857475989468986524396117858372447166857367242140321403352265023767940092764835400407396491849744178163839268602181644683845454415964714692287070164562732005152443153934934569784873638491039906159857965911863668685595012796157381760638400887063762179937320188917875309314905688674605533327080917687085535959369688035268078

Below, we display the public key (consisting of $e$ and $n$) and the private key (with $d$ and $n$).

In [249]:
# Display the generated keys
print("=== RSA Key Generation ===")
print("Public Key (e, n):")
print("e =", e)
print("n =", n)
print("\nPrivate Key (d, n):")
print("d =", d)

=== RSA Key Generation ===
Public Key (e, n):
e = 65537
n = 3868332360874582027346128505197869306107240528630347688505856925602299370439949850854511454484669325186705044211516209956483219619376109998109495613140903663554426903477220924612213845910069775222084435398044418118668080721041477489387380376793655639905844190652497568298769411802727650615877698004442953124743536633634120023288917414384927006969888499016969055304736961267930666948609899264304404096191369993905876159860885557525959681057844316960973188522827402388996941971797619370398222368087671995808242829032017464865996647532743778925076070597689219152299747567563328869601373300563632387225956895137838830558560583572852560763874914823654714973149057155235560840139889176643928366062768564200710172422621703845638687485337145684189642280566443286706755246635553155116087541468820801826482540791539452978156815995683505352891440680997054841952488527482458148315336988988707104692662818069455979958895640133818822076311831369961603171

### Primality Testing

This code implements the Miller-Rabin primality test. The function "power" efficiently computes modular exponentiation, and "strong_pseudoprimality_test" runs rounds of testing with random bases. If a chosen base fails to confirm a number as a strong pseudoprime, the function returns "composite"; otherwise, it concludes that the given number is a strong pseudoprime. The purpose is to confirm that $p$ and $q$ are safe primes.

In [253]:
import random

def power(a, d, n):
    result = 1
    a %= n
    while d > 0:
        if d % 2 == 1:
            result = (result * a) % n
        a = (a * a) % n
        d //= 2
    return result

def strong_pseudoprimality_test(n, k):
    if n == 2 or n == 3:
        return "strong pseudoprime"
    if n <= 1 or n % 2 == 0:
        return "composite"

    # Find (s, r) pair such that n - 1 = 2**s * r
    s = 0
    r = n - 1
    while r % 2 == 0:
        s += 1
        r //= 2

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = power(a, r, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = power(x, 2, n)
            if x == n - 1:
                break
        else:
            # Test failed
            return "composite"
    return "strong pseudoprime" # Test passed

In [254]:
# Verifying the primality of p and q
k = 30 # Number of iterations
print("p is a", strong_pseudoprimality_test(p, k))
print("q is a", strong_pseudoprimality_test(q, k))

p is a strong pseudoprime
q is a strong pseudoprime


### Encrypting a Message

Here, we convert a numeric string representing the message into an integer, then encrypt it using RSA by computing $c = m^e \mod n$ (with previously computed $e$ and $n$).

In [256]:
# Your given message value as a numeric string:
message_str = "1819370023350024112315001929002025182400112414001900112300172211140035253100181132150014151325141514003018150023152929111715380019002031293000331124300030250033192918003525310033152222001129003525310017250011122531300030181500232524301800251600231128131838"


# Convert the message string to an integer
m_int = int(message_str)

# Encrypt the message using RSA encryption: ciphertext = m^e mod n
# (Assuming e and n have already been computed)
ciphertext = pow(m_int, e, n)

print("Encrypted ciphertext:")
print(ciphertext)


Encrypted ciphertext:
133123132256505212203474499063979092107547536898253259238878904608663275562746961871619361180963650741701610308803026814758562312084734709888259447513920029442899349095343828436070053247385352805599469643927933923310123779511142730871040872507566040256305388479099579189125453890652401549565388664440772728770582738067810238858610802796795655445223022446691792515326295896540252497937011540289110282927698484513627801483770631376130531359390956316326950245750748420939009334738722496398422954137710164668918249757145787530489063578703722264703376393621900383661251734317443918352726869244558493035490602900378349434773192863271407707139069711706363053865155621560853028664691738130767621145771357761304995481851356684194120152275042832598487829504253819327828574886297524153888344863782036073806554085887006035027529929992412091210162484618992255952938017551580192234830779137359120411907908383272293652341929810389534366418284884460576461574409638638208768670053601950429639784

### Decrypting the Encoded Message

We go on to decrypt the RSA ciphertext by computing $ m = c^d \mod n $, convert the resulting integer into a string, and print the decrypted message.

In [258]:
# Decrypt the ciphertext using RSA decryption: m = c^d mod n
decrypted_message_int = pow(ciphertext, d, n)
decrypted_message_str = str(decrypted_message_int)
print("Decrypted message (as string):", decrypted_message_str)

Decrypted message (as string): 1819370023350024112315001929002025182400112414001900112300172211140035253100181132150014151325141514003018150023152929111715380019002031293000331124300030250033192918003525310033152222001129003525310017250011122531300030181500232524301800251600231128131838


Finally, we define a mapping from two-digit codes to characters, converts the decrypted integer into a string (padding with a leading zero if needed), splits it into two-digit groups, translates each group using the mapping, and prints the resulting decoded message.

In [259]:
# Define the custom mapping for two-digit codes
char_map = {
    "00": " ",
    "11": "A", "12": "B", "13": "C", "14": "D", "15": "E", "16": "F",
    "17": "G", "18": "H", "19": "I", "20": "J", "21": "K", "22": "L",
    "23": "M", "24": "N", "25": "O", "26": "P", "27": "Q", "28": "R",
    "29": "S", "30": "T", "31": "U", "32": "V", "33": "W", "34": "X",
    "35": "Y", "36": "Z", "37": ",", "38": "."
}

# Convert the decrypted integer to a string.
decrypted_str = str(decrypted_message_int)

# Ensure the string has an even number of digits; pad with a leading zero if necessary.
if len(decrypted_str) % 2 != 0:
    decrypted_str = "0" + decrypted_str

# Decode the message by processing two digits at a time.
decoded_message = ""
for i in range(0, len(decrypted_str), 2):
    # Extract a two-digit code
    code = decrypted_str[i:i+2]
    # Lookup the corresponding character; use '?' if the code is not found
    decoded_message += char_map.get(code, "?")

print("Decoded message:", decoded_message)


Decoded message: HI, MY NAME IS JOHN AND I AM GLAD YOU HAVE DECODED THE MESSAGE. I JUST WANT TO WISH YOU WELL AS YOU GO ABOUT THE MONTH OF MARCH.


## Conclusion
We have now successfully implemented the entire RSA process. We generated 600-digit safe primes, computed the RSA modulus and Euler’s totient function, and derived both the public and private keys. Using these keys, we encrypted a numeric message and then decrypted it back to its original encoded form. Finally, by applying our custom two-digit mapping, we decoded the message to reveal:

**HI, MY NAME IS JOHN AND I AM GLAD YOU HAVE DECODED THE MESSAGE. I JUST WANT TO WISH YOU WELL AS YOU GO ABOUT THE MONTH OF MARCH.**